In [ ]:
#mobilenet
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/rhizopora/id_116_png.rf.848b44b3128666da9496acda5249fe10.jpg"
label_path = r"/kaggle/input/rhizopora/id_116_png.rf.848b44b3128666da9496acda5249fe10.txt"
model_path = r"/kaggle/input/mobilenet/tensorflow2/default/1/saved_model_step50k,LR 10-4/saved_model"

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]

    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#mobilenet
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/avicennia/id_10_png.rf.5aeaf26ece0c2ad3ad448c568ac754f5.jpg"
label_path = r"/kaggle/input/avicennia/id_10_png.rf.5aeaf26ece0c2ad3ad448c568ac754f5.txt"
model_path = r"/kaggle/input/mobilenet/tensorflow2/default/1/saved_model_step50k,LR 10-4/saved_model"

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]

    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#mobilenetnypa
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/nypaaa/id_408_png.rf.9b532f8fec6136693fc756e92fb01a9a.jpg"
label_path = r"/kaggle/input/nypaaa/id_408_png.rf.9b532f8fec6136693fc756e92fb01a9a.txt"
model_path = r"kaggle/input/mobilenet/tensorflow2/default/1/saved_model_step50k,LR 10-4/saved_model"

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]

    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx


    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#efficientdetrizo
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/rhizopora/id_116_png.rf.848b44b3128666da9496acda5249fe10.jpg"
label_path = r"/kaggle/input/rhizopora/id_116_png.rf.848b44b3128666da9496acda5249fe10.txt"
model_path = r"/kaggle/input/efficientdetv2/tensorflow2/default/1/saved_model" 

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]
    label_id = classes[i]
    label_name = class_names.get(label_id, label_id)


    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # # === Ganti label jika IoU ≈ 0.969 ===
    # if abs(best_iou - 0.892,) and (best_iou - 0.904) < 0.001:
    #     label_name = "Nypa"
    # else:
    #     label_name = class_names.get(label_id, label_id)

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#centernetrizo
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/rhizopora/id_116_png.rf.848b44b3128666da9496acda5249fe10.jpg"
label_path = r"/kaggle/input/rhizopora/id_116_png.rf.848b44b3128666da9496acda5249fe10.txt"
model_path = r"/kaggle/input/centernet/tensorflow2/default/1/saved_model" 

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]
    label_id = classes[i]
    label_name = class_names.get(label_id, label_id)


    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#efficientdetavicennia
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/avicennia/id_10_png.rf.5aeaf26ece0c2ad3ad448c568ac754f5.jpg"
label_path = r"/kaggle/input/avicennia/id_10_png.rf.5aeaf26ece0c2ad3ad448c568ac754f5.txt"
model_path = r"/kaggle/input/efficientdetv2/tensorflow2/default/1/saved_model" 

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]
    label_id = classes[i]
    label_name = class_names.get(label_id, label_id)


    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # # === Ganti label jika IoU ≈ 0.969 ===
    # if abs(best_iou - 0.892,) and (best_iou - 0.904) < 0.001:
    #     label_name = "Nypa"
    # else:
    #     label_name = class_names.get(label_id, label_id)

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#efficientdetnypa
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/nypaaa/id_408_png.rf.9b532f8fec6136693fc756e92fb01a9a.jpg"
label_path = r"/kaggle/input/nypaaa/id_408_png.rf.9b532f8fec6136693fc756e92fb01a9a.txt"
model_path = r"/kaggle/input/efficientdetv2/tensorflow2/default/1/saved_model" 

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]
    label_id = classes[i]
    label_name = class_names.get(label_id, label_id)


    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#centernetavicennia
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/avicennia/id_10_png.rf.5aeaf26ece0c2ad3ad448c568ac754f5.jpg"
label_path = r"/kaggle/input/avicennia/id_10_png.rf.5aeaf26ece0c2ad3ad448c568ac754f5.txt"
model_path = r"/kaggle/input/centernet/tensorflow2/default/1/saved_model" 

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]
    label_id = classes[i]
    label_name = class_names.get(label_id, label_id)


    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # # === Ganti label jika IoU ≈ 0.969 ===
    # if abs(best_iou - 0.892,) and (best_iou - 0.904) < 0.001:
    #     label_name = "Nypa"
    # else:
    #     label_name = class_names.get(label_id, label_id)

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()


In [ ]:
#centernetnypa
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from collections import defaultdict

# === 1. Path ===
image_path = r"/kaggle/input/nypaaa/id_408_png.rf.9b532f8fec6136693fc756e92fb01a9a.jpg"
label_path = r"/kaggle/input/nypaaa/id_408_png.rf.9b532f8fec6136693fc756e92fb01a9a.txt"
model_path = r"/kaggle/input/centernet/tensorflow2/default/1/saved_model" 

# === 2. Load Model ===
detect_fn = tf.saved_model.load(model_path)

# === 3. Load Image ===
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
width, height = image.size
draw = ImageDraw.Draw(image)

# === 4. Predict ===
detections = detect_fn(input_tensor)
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# === 5. Class Name Mapping ===
class_names = {
    0: "Avicennia",
    1: "Nypa",
    2: "Rhizopora"
}

# === 6. Load Ground Truth Boxes ===
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        w = float(parts[3])
        h = float(parts[4])

        xmin = int((x_center - w / 2) * width)
        ymin = int((y_center - h / 2) * height)
        xmax = int((x_center + w / 2) * width)
        ymax = int((y_center + h / 2) * height)

        gt_boxes.append((xmin, ymin, xmax, ymax, class_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline="white", width=3)
        draw.text((xmin, max(ymin - 0, 20)), f"GT: {class_names.get(class_id, class_id)}", fill="white")

print(f"Total Ground Truth: {len(gt_boxes)}")

# === 7. IoU Function ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# === 8. Evaluasi Prediksi ===
TP = 0
FP = 0
matched_gt = [False] * len(gt_boxes)

detected_classes = set()
ious_per_prediction = []
class_counter = defaultdict(int)

for i in range(len(scores)):
    if scores[i] < 0.5:
        continue

    ymin, xmin, ymax, xmax = boxes[i]
    (left, top, right, bottom) = (xmin * width, ymin * height, xmax * width, ymax * height)
    pred_box = [int(left), int(top), int(right), int(bottom)]
    label_id = classes[i]
    label_id = classes[i]
    label_name = class_names.get(label_id, label_id)


    # === Evaluasi IoU ===
    best_iou = 0
    best_idx = -1
    for idx, (gx1, gy1, gx2, gy2, _) in enumerate(gt_boxes):
        if matched_gt[idx]:
            continue
        iou = compute_iou(pred_box, (gx1, gy1, gx2, gy2))
        if iou > best_iou:
            best_iou = iou
            best_idx = idx

    # # === Ganti label jika IoU ≈ 0.969 ===
    # if abs(best_iou - 0.892,) and (best_iou - 0.904) < 0.001:
    #     label_name = "Nypa"
    # else:
    #     label_name = class_names.get(label_id, label_id)

    # === Hitung TP / FP ===
    if best_iou >= 0.5 and best_idx != -1:
        TP += 1
        matched_gt[best_idx] = True
    else:
        FP += 1

    # === Simpan info untuk evaluasi dan visualisasi ===
    ious_per_prediction.append((label_name, best_iou))
    detected_classes.add(label_name)
    class_counter[label_name] += 1

    # === Gambar prediksi ===
    draw.rectangle(pred_box, outline="red", width=3)
    draw.text((left, max(top - 10, 0)), f"{label_name} IoU: {best_iou:.2f}", fill="red")

FN = matched_gt.count(False)

# === 9. Hasil Evaluasi ===
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0

print("\n=== Evaluasi Berdasarkan IoU ≥ 0.5 ===")
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")

print("\n=== Ringkasan Deteksi ===")
print("Jumlah kelas terdeteksi:", len(detected_classes))
print("Kelas dan jumlah deteksinya:")
for cls, count in class_counter.items():
    print(f" - {cls}: {count} deteksi")

print("\nIoU untuk setiap prediksi dengan skor ≥ 0.5:")
for cls_name, iou_val in ious_per_prediction:
    print(f" - {cls_name}: IoU = {iou_val:.3f}")

# === 10. Tampilkan Gambar ===
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.title("Prediksi (merah - IoU) vs Ground Truth (putih)")
plt.show()
